# Ungraded Lab: Build a Multi-output Model

In this lab, we'll show how you can build models with more than one output. The dataset we will be working on is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It is an Energy Efficiency dataset which uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. Let's see how we can build a model to train on this data.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Utilities

We define a few utilities for data conversion and visualization to make our code more neat.

In [2]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

## Prepare the Data

We download the dataset and format it for training.

In [3]:
# Get the data from UCI dataset
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'

# Use pandas excel reader
df = pd.read_excel(URL)
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [4]:
# Define model layers.
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
______________________________________________________________________________________________

## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [5]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the Model

In [ ]:
# Train the model for 500 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 1s 892us/sample - loss: 260.9925 - y1_output_loss: 116.7876 - y2_output_loss: 141.9153 - y1_output_root_mean_squared_error: 10.8558 - y2_output_root_mean_squared_error: 11.9643 - val_loss: 82.5837 - val_y1_output_loss: 20.0027 - val_y2_output_loss: 61.4736 - val_y1_output_root_mean_squared_error: 4.5403 - val_y2_output_root_mean_squared_error: 7.8721
Epoch 2/500
614/614 [==============================] - 0s 140us/sample - loss: 35.4986 - y1_output_loss: 12.8690 - y2_output_loss: 22.3870 - y1_output_root_mean_squared_error: 3.5981 - y2_output_root_mean_squared_error: 4.7490 - val_loss: 23.2949 - val_y1_output_loss: 10.6772 - val_y2_output_loss: 11.9706 - val_y1_output_root_mean_squared_error: 3.3233 - val_y2_output_root_mean_squared_error: 3.5001
Epoch 3/500
614/614 [==============================] - 0s 138us/sample - loss: 27.4191 - y1_output_loss: 10.0831 - y2_output_loss: 17.6095 - y1

Epoch 42/500
614/614 [==============================] - 0s 123us/sample - loss: 7.1896 - y1_output_loss: 2.3132 - y2_output_loss: 4.8317 - y1_output_root_mean_squared_error: 1.5258 - y2_output_root_mean_squared_error: 2.2049 - val_loss: 7.4706 - val_y1_output_loss: 2.5824 - val_y2_output_loss: 4.6590 - val_y1_output_root_mean_squared_error: 1.6351 - val_y2_output_root_mean_squared_error: 2.1902
Epoch 43/500
614/614 [==============================] - 0s 125us/sample - loss: 5.5935 - y1_output_loss: 1.8906 - y2_output_loss: 3.6908 - y1_output_root_mean_squared_error: 1.3756 - y2_output_root_mean_squared_error: 1.9238 - val_loss: 16.6388 - val_y1_output_loss: 4.4806 - val_y2_output_loss: 12.0148 - val_y1_output_root_mean_squared_error: 2.1165 - val_y2_output_root_mean_squared_error: 3.4870
Epoch 44/500
614/614 [==============================] - 0s 128us/sample - loss: 7.3545 - y1_output_loss: 2.2412 - y2_output_loss: 5.0777 - y1_output_root_mean_squared_error: 1.4963 - y2_output_root_mean

Epoch 63/500
614/614 [==============================] - 0s 133us/sample - loss: 6.1396 - y1_output_loss: 1.6155 - y2_output_loss: 4.5055 - y1_output_root_mean_squared_error: 1.2630 - y2_output_root_mean_squared_error: 2.1318 - val_loss: 5.2667 - val_y1_output_loss: 1.7257 - val_y2_output_loss: 3.3918 - val_y1_output_root_mean_squared_error: 1.3365 - val_y2_output_root_mean_squared_error: 1.8656
Epoch 64/500
614/614 [==============================] - 0s 122us/sample - loss: 8.7294 - y1_output_loss: 2.6124 - y2_output_loss: 6.1747 - y1_output_root_mean_squared_error: 1.5990 - y2_output_root_mean_squared_error: 2.4845 - val_loss: 20.0583 - val_y1_output_loss: 6.8169 - val_y2_output_loss: 13.2813 - val_y1_output_root_mean_squared_error: 2.6135 - val_y2_output_root_mean_squared_error: 3.6370
Epoch 65/500
614/614 [==============================] - 0s 123us/sample - loss: 6.0095 - y1_output_loss: 1.7543 - y2_output_loss: 4.2298 - y1_output_root_mean_squared_error: 1.3245 - y2_output_root_mean

Epoch 84/500
614/614 [==============================] - 0s 124us/sample - loss: 3.6399 - y1_output_loss: 0.9942 - y2_output_loss: 2.6888 - y1_output_root_mean_squared_error: 1.0009 - y2_output_root_mean_squared_error: 1.6242 - val_loss: 9.1907 - val_y1_output_loss: 1.2907 - val_y2_output_loss: 7.8408 - val_y1_output_root_mean_squared_error: 1.1463 - val_y2_output_root_mean_squared_error: 2.8066
Epoch 85/500
614/614 [==============================] - 0s 123us/sample - loss: 3.8542 - y1_output_loss: 1.0753 - y2_output_loss: 2.7456 - y1_output_root_mean_squared_error: 1.0413 - y2_output_root_mean_squared_error: 1.6643 - val_loss: 3.1412 - val_y1_output_loss: 0.7427 - val_y2_output_loss: 2.3073 - val_y1_output_root_mean_squared_error: 0.8681 - val_y2_output_root_mean_squared_error: 1.5452
Epoch 86/500
614/614 [==============================] - 0s 125us/sample - loss: 4.3390 - y1_output_loss: 1.0192 - y2_output_loss: 3.2872 - y1_output_root_mean_squared_error: 1.0114 - y2_output_root_mean_s

Epoch 105/500
614/614 [==============================] - 0s 125us/sample - loss: 2.4680 - y1_output_loss: 0.5632 - y2_output_loss: 1.9087 - y1_output_root_mean_squared_error: 0.7521 - y2_output_root_mean_squared_error: 1.3792 - val_loss: 3.8807 - val_y1_output_loss: 1.1356 - val_y2_output_loss: 2.6894 - val_y1_output_root_mean_squared_error: 1.0586 - val_y2_output_root_mean_squared_error: 1.6613
Epoch 106/500
614/614 [==============================] - 0s 121us/sample - loss: 3.1906 - y1_output_loss: 0.6891 - y2_output_loss: 2.4734 - y1_output_root_mean_squared_error: 0.8332 - y2_output_root_mean_squared_error: 1.5800 - val_loss: 2.4339 - val_y1_output_loss: 0.4372 - val_y2_output_loss: 1.9151 - val_y1_output_root_mean_squared_error: 0.6692 - val_y2_output_root_mean_squared_error: 1.4093
Epoch 107/500
614/614 [==============================] - 0s 128us/sample - loss: 3.8765 - y1_output_loss: 1.0976 - y2_output_loss: 2.7577 - y1_output_root_mean_squared_error: 1.0494 - y2_output_root_mea

...

Epoch 187/500
614/614 [==============================] - 0s 123us/sample - loss: 2.0321 - y1_output_loss: 0.5013 - y2_output_loss: 1.5527 - y1_output_root_mean_squared_error: 0.7103 - y2_output_root_mean_squared_error: 1.2359 - val_loss: 5.7954 - val_y1_output_loss: 0.6970 - val_y2_output_loss: 4.9702 - val_y1_output_root_mean_squared_error: 0.8339 - val_y2_output_root_mean_squared_error: 2.2583
Epoch 188/500
614/614 [==============================] - 0s 123us/sample - loss: 5.7411 - y1_output_loss: 1.3531 - y2_output_loss: 4.3419 - y1_output_root_mean_squared_error: 1.1689 - y2_output_root_mean_squared_error: 2.0916 - val_loss: 2.9968 - val_y1_output_loss: 0.5578 - val_y2_output_loss: 2.4241 - val_y1_output_root_mean_squared_error: 0.7298 - val_y2_output_root_mean_squared_error: 1.5698
Epoch 189/500
614/614 [==============================] - 0s 123us/sample - loss: 2.3904 - y1_output_loss: 0.5060 - y2_output_loss: 1.8657 - y1_output_root_mean_squared_error: 0.7134 - y2_output_root_mea

Epoch 228/500
614/614 [==============================] - 0s 137us/sample - loss: 1.0602 - y1_output_loss: 0.2210 - y2_output_loss: 0.8320 - y1_output_root_mean_squared_error: 0.4705 - y2_output_root_mean_squared_error: 0.9159 - val_loss: 1.2908 - val_y1_output_loss: 0.2101 - val_y2_output_loss: 1.0377 - val_y1_output_root_mean_squared_error: 0.4645 - val_y2_output_root_mean_squared_error: 1.0369
Epoch 229/500
614/614 [==============================] - 0s 134us/sample - loss: 1.2878 - y1_output_loss: 0.2610 - y2_output_loss: 1.0198 - y1_output_root_mean_squared_error: 0.5107 - y2_output_root_mean_squared_error: 1.0134 - val_loss: 1.3989 - val_y1_output_loss: 0.1836 - val_y2_output_loss: 1.1692 - val_y1_output_root_mean_squared_error: 0.4331 - val_y2_output_root_mean_squared_error: 1.1006
Epoch 230/500
614/614 [==============================] - 0s 132us/sample - loss: 1.2913 - y1_output_loss: 0.2857 - y2_output_loss: 0.9964 - y1_output_root_mean_squared_error: 0.5348 - y2_output_root_mea

Epoch 269/500
614/614 [==============================] - 0s 137us/sample - loss: 1.2725 - y1_output_loss: 0.3071 - y2_output_loss: 0.9567 - y1_output_root_mean_squared_error: 0.5564 - y2_output_root_mean_squared_error: 0.9813 - val_loss: 1.5348 - val_y1_output_loss: 0.4340 - val_y2_output_loss: 1.0828 - val_y1_output_root_mean_squared_error: 0.6656 - val_y2_output_root_mean_squared_error: 1.0449
Epoch 270/500
614/614 [==============================] - 0s 132us/sample - loss: 1.0837 - y1_output_loss: 0.2359 - y2_output_loss: 0.8411 - y1_output_root_mean_squared_error: 0.4870 - y2_output_root_mean_squared_error: 0.9201 - val_loss: 1.0702 - val_y1_output_loss: 0.2153 - val_y2_output_loss: 0.8355 - val_y1_output_root_mean_squared_error: 0.4674 - val_y2_output_root_mean_squared_error: 0.9229
Epoch 271/500
614/614 [==============================] - 0s 128us/sample - loss: 0.9222 - y1_output_loss: 0.2164 - y2_output_loss: 0.6984 - y1_output_root_mean_squared_error: 0.4667 - y2_output_root_mea

Epoch 310/500
614/614 [==============================] - 0s 122us/sample - loss: 0.9760 - y1_output_loss: 0.2396 - y2_output_loss: 0.7631 - y1_output_root_mean_squared_error: 0.4911 - y2_output_root_mean_squared_error: 0.8572 - val_loss: 5.7949 - val_y1_output_loss: 0.3275 - val_y2_output_loss: 5.3334 - val_y1_output_root_mean_squared_error: 0.5761 - val_y2_output_root_mean_squared_error: 2.3373
Epoch 311/500
614/614 [==============================] - 0s 121us/sample - loss: 0.8141 - y1_output_loss: 0.1786 - y2_output_loss: 0.6325 - y1_output_root_mean_squared_error: 0.4245 - y2_output_root_mean_squared_error: 0.7962 - val_loss: 1.0567 - val_y1_output_loss: 0.2162 - val_y2_output_loss: 0.8265 - val_y1_output_root_mean_squared_error: 0.4723 - val_y2_output_root_mean_squared_error: 0.9130
Epoch 312/500
614/614 [==============================] - 0s 120us/sample - loss: 0.7691 - y1_output_loss: 0.1820 - y2_output_loss: 0.5814 - y1_output_root_mean_squared_error: 0.4280 - y2_output_root_mea

Epoch 351/500
614/614 [==============================] - 0s 132us/sample - loss: 0.6485 - y1_output_loss: 0.1655 - y2_output_loss: 0.4847 - y1_output_root_mean_squared_error: 0.4013 - y2_output_root_mean_squared_error: 0.6982 - val_loss: 1.2964 - val_y1_output_loss: 0.1973 - val_y2_output_loss: 1.0751 - val_y1_output_root_mean_squared_error: 0.4369 - val_y2_output_root_mean_squared_error: 1.0514
Epoch 352/500
614/614 [==============================] - 0s 137us/sample - loss: 0.8089 - y1_output_loss: 0.1579 - y2_output_loss: 0.6685 - y1_output_root_mean_squared_error: 0.3975 - y2_output_root_mean_squared_error: 0.8068 - val_loss: 2.9839 - val_y1_output_loss: 0.4691 - val_y2_output_loss: 2.4190 - val_y1_output_root_mean_squared_error: 0.6967 - val_y2_output_root_mean_squared_error: 1.5807
Epoch 353/500
614/614 [==============================] - 0s 131us/sample - loss: 1.0416 - y1_output_loss: 0.2685 - y2_output_loss: 0.7659 - y1_output_root_mean_squared_error: 0.5190 - y2_output_root_mea

## Evaluate the Model and Plot Metrics

In [ ]:
# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)